In [1]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mnatali/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
import pandas as pd
df = pd.read_csv("data/test.csv", encoding = "latin-1")

In [7]:
df = df.dropna(subset = ["text"])
df = df[["text", "sentiment"]]
df.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [8]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
def vader_score(text):
    return sia.polarity_scores(text)
df["vader"]= df["text"].apply(vader_score)
df["compound"] = df["vader"].apply(lambda x: x["compound"])

In [9]:
def vader_label(compound):
    if compound >= 0.05:
        return "positive"
    elif compound <= -0.05:
        return "negative"
    else:
        return "neutral"
df["label"] = df["compound"].apply(vader_label)
df.head()

,text,sentiment,vader,compound,label
0,Last session of the day http://twitpic.com/67ezh,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
1,Shanghai is also really exciting (precisely -...,positive,"{'neg': 0.0, 'neu': 0.67, 'pos': 0.33, 'compou...",0.7501,positive
2,"Recession hit Veronique Branquinho, she has to...",negative,"{'neg': 0.382, 'neu': 0.618, 'pos': 0.0, 'comp...",-0.7345,negative
3,happy bday!,positive,"{'neg': 0.0, 'neu': 0.2, 'pos': 0.8, 'compound...",0.6114,positive
4,http://twitpic.com/4w75p - I like it!!,positive,"{'neg': 0.0, 'neu': 0.393, 'pos': 0.607, 'comp...",0.4738,positive


In [10]:
##comparing it to original dataset classification
TP = FP = TN = FN = T0 = F0 = 0
def compare(row):
    global TP, FP, TN, FN, T0, F0

    trueVal = row["sentiment"]
    vaderVal = row["label"]

    if vaderVal == "positive":
        if vaderVal == trueVal:
            TP += 1
        else:
            FP += 1
    elif vaderVal == "negative":
        if vaderVal == trueVal:
            TN += 1
        else:
            FN += 1
    else:
        if vaderVal == trueVal:
            T0 += 1
        else:
            F0 += 1

df.apply(compare, axis = 1)

accuracy = (TP+TN+T0)/(TP+FP+TN+FN+T0+F0)
positivePrecision = TP/(TP + FP)
negativePrecision = TN/(TN + FN)
neutralPrecision = T0/(T0 + F0)

print("Accuracy: ", accuracy)

Accuracy:  0.6479909451046972


In [13]:
mistakeDF = df[df["sentiment"] != df["label"]].copy()

mistakeDF.head(15)

,text,sentiment,vader,compound,label
11,I checked. We didn`t win,neutral,"{'neg': 0.0, 'neu': 0.441, 'pos': 0.559, 'comp...",0.5859,positive
12,.. and you`re on twitter! Did the tavern bore...,neutral,"{'neg': 0.172, 'neu': 0.828, 'pos': 0.0, 'comp...",-0.3164,negative
14,Its coming out the socket I feel like my phon...,negative,"{'neg': 0.106, 'neu': 0.78, 'pos': 0.115, 'com...",0.0516,positive
17,Cramps . . .,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
19,"I`m going into a spiritual stagnentation, its ...",neutral,"{'neg': 0.145, 'neu': 0.767, 'pos': 0.089, 'co...",-0.3369,negative
26,"hey peoples, dont you just hate being grounded...",neutral,"{'neg': 0.14, 'neu': 0.645, 'pos': 0.215, 'com...",0.2728,positive
28,look who I found just for you ---> http://t...,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
29,"No AC, the fan doesnt swing our way ... we are...",negative,"{'neg': 0.107, 'neu': 0.78, 'pos': 0.112, 'com...",0.0258,neutral
30,guess that depends on if you want to be on th...,neutral,"{'neg': 0.0, 'neu': 0.894, 'pos': 0.106, 'comp...",0.0772,positive
31,resorted to eating Mickey Ds ALONE.,neutral,"{'neg': 0.353, 'neu': 0.647, 'pos': 0.0, 'comp...",-0.4084,negative











Test Dataset #2:

In [20]:
from datasets import load_dataset

ds = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")
ds = ds["train"]

In [22]:
df2 = ds.to_pandas()
df2.head()

,id,text,label,sentiment
0,9536,"Cooking microwave pizzas, yummy",2,positive
1,6135,Any plans of allowing sub tasks to show up in ...,1,neutral
2,17697,"I love the humor, I just reworded it. Like sa...",2,positive
3,14182,naw idk what ur talkin about,1,neutral
4,17840,That sucks to hear. I hate days like that,0,negative


In [24]:
df2["vader"]= df2["text"].apply(vader_score)
df2["compound"] = df2["vader"].apply(lambda x: x["compound"])
df2["label"] = df2["compound"].apply(vader_label)
df.head()

,text,sentiment,vader,compound,label
0,Last session of the day http://twitpic.com/67ezh,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
1,Shanghai is also really exciting (precisely -...,positive,"{'neg': 0.0, 'neu': 0.67, 'pos': 0.33, 'compou...",0.7501,positive
2,"Recession hit Veronique Branquinho, she has to...",negative,"{'neg': 0.382, 'neu': 0.618, 'pos': 0.0, 'comp...",-0.7345,negative
3,happy bday!,positive,"{'neg': 0.0, 'neu': 0.2, 'pos': 0.8, 'compound...",0.6114,positive
4,http://twitpic.com/4w75p - I like it!!,positive,"{'neg': 0.0, 'neu': 0.393, 'pos': 0.607, 'comp...",0.4738,positive


In [26]:
TP = FP = TN = FN = T0 = F0 = 0
df2.apply(compare, axis = 1)

accuracy = (TP+TN+T0)/(TP+FP+TN+FN+T0+F0)
positivePrecision = TP/(TP + FP)
negativePrecision = TN/(TN + FN)
neutralPrecision = T0/(T0 + F0)

print("Accuracy: ", accuracy)

mistakeDF2 = df2[df2["sentiment"] != df2["label"]].copy()

mistakeDF2.head(15)

Accuracy:  0.610687756147541


,id,text,label,sentiment,vader,compound
3,14182,naw idk what ur talkin about,negative,neutral,"{'neg': 0.219, 'neu': 0.781, 'pos': 0.0, 'comp...",-0.1027
12,1399,what did you think of the finale? I thought ...,neutral,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
15,8340,Installed office for Mac & messed up all my fo...,positive,negative,"{'neg': 0.091, 'neu': 0.72, 'pos': 0.189, 'com...",0.3786
16,7922,You`re welcome...,neutral,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
17,23219,Ok so I`m **** hungry....! I`m fat I know..I ...,positive,negative,"{'neg': 0.0, 'neu': 0.764, 'pos': 0.236, 'comp...",0.4184
18,9289,Early monday cramming... yay. Only a few weeks...,positive,neutral,"{'neg': 0.0, 'neu': 0.673, 'pos': 0.327, 'comp...",0.5267
19,18568,ok its FF soooo why isn`t anyone following MEE...,positive,neutral,"{'neg': 0.151, 'neu': 0.539, 'pos': 0.311, 'co...",0.7226
29,2290,"Guess I`m gonna try the nap thing again 2day, ...",positive,negative,"{'neg': 0.0, 'neu': 0.894, 'pos': 0.106, 'comp...",0.4497
34,17442,shall bring his own glass to work from tonight...,negative,neutral,"{'neg': 0.176, 'neu': 0.824, 'pos': 0.0, 'comp...",-0.4184
35,6000,Not as useful as Wunderlist... Yet!! Would pre...,positive,neutral,"{'neg': 0.071, 'neu': 0.831, 'pos': 0.097, 'co...",0.2211
